In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/sample_submission.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/engine_metadata.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/site_metadata.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_141.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_202.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_26.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_165.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_124.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_178.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_33.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_49.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_79.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data

# Code for the Kaggle score leaderboard

## Load Data

In [2]:
# Load engine_metadata
engine_metadata = pd.read_csv("/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/engine_metadata.csv")

In [3]:
# Searching ancient wasp
engine_metadata["PLANT_NAME"].unique()

array(['SPIRITUAL-POLECAT', 'NIFTY-ROOK', 'PREHISTORIC-PETREL',
       'THERAPEUTIC-LIONFISH', 'ABORIGINAL-PICULET', 'CARMINE-REINDEER',
       'IDEALISTIC-DODO', 'ANCIENT-WASP', 'AQUAMARINE-KANGAROO',
       'RADIANT-STURGEON', 'CURIOUS-QUAIL', 'SILKY-WEASEL',
       'INDIGO-TAPIR', 'PERFECT-MACKEREL', 'ADORABLE-PANTHER',
       'CAREFUL-PHEASANT', 'PRECISE-POGONA', 'FIERY-BOBCAT',
       'BIPEDAL-UAKARI', 'LUSH-CHIPMUNK', 'HYPERSONIC-BULLDOG',
       'TUSCAN-CAPUCHIN', 'MUSTARD-PANDA', 'NIFTY-MULE',
       'TOURMALINE-MOUSE', 'ANNOYING-SPONGE', 'ROSE-KINGFISHER',
       'BEAUTIFUL-HIPPO', 'VORACIOUS-PARROT', 'STOIC-KOEL',
       'COBALT-CATFISH', 'CERULEAN-NAUTILUS', 'PRAGMATIC-MASTIFF',
       'RAPID-CARACARA', 'COBALT-MALKOHA', 'SAGE-PRAWN',
       'ANGELIC-REINDEER', 'INESCAPABLE-QUOKKA', 'DIDACTIC-HORNET',
       'SINGING-HONEYBEE', 'LOUD-OCELOT', 'LOUTISH-PANDA', 'ZIRCON-CAMEL',
       'GIGANTIC-EARWIG', 'PREMIUM-MOTH'], dtype=object)

In [4]:
# Searching for the data file
engine_metadata[(engine_metadata["PLANT_NAME"] == "ANCIENT-WASP") & (engine_metadata["ENGINE_ID"] == "ENGINE_3")]

,CUSTOMER_NAME,PLANT_NAME,ENGINE_ID,FILE_ID
32,BUSY,ANCIENT-WASP,ENGINE_3,data_33.csv


In [5]:
# Loading Ancient_wasp file
Ancient_wasp_engine3 = pd.read_csv("/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_33.csv")

In [6]:
# viewing the dataframe
Ancient_wasp_engine3.head()

,Unnamed: 0,CMP_SPEED,POWER,FUEL_FLOW,CO2
0,01/01/2021 00:00,5589.869257,4900.793536,0.451669,1.229985
1,01/01/2021 01:00,5586.765793,4879.848427,0.450786,1.227582
2,01/01/2021 02:00,5599.140667,4902.774026,0.451686,1.230033
3,01/01/2021 03:00,5601.607464,4902.992111,0.451230,1.228792
4,01/01/2021 04:00,5573.922031,4878.853341,0.450572,1.227000


## Reviewing format for the deliverable

In [7]:
# loading the sample
sample_submission = pd.read_csv("/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/sample_submission.csv")

# seeing the end of the dataframe
sample_submission.tail()

,ID,HOURS
8756,8756,6624
8757,8757,6625
8758,8758,6626
8759,8759,6627
8760,8760,6628


# Creating the hour counter

ID is a list of numbers from 0 to 8760, it corresponds to the number of rows in the dataframe.

HOURS is an empty list where the number of hours will be added in ascending order.

In [8]:
ID = [i for i in range(8761)]
HOURS = []

### Loop to review all the rows that exist and add an hour to the HOURS list.

In [9]:
# loop through the number of rows
for i in ID:
    
    # Creating a try loop to stop the error that occurs from reaching the end of the dataframe
    try:
        
        # 0 when it is the first row
        if Ancient_wasp_engine3.index[i] == 0:
            HOURS.append(0)
        # we add 1 to the previous number in case the CMP_SPEED is greater than 0
        elif (Ancient_wasp_engine3.loc[i, "CMP_SPEED"] > 0):
            HOURS.append(HOURS[-1] + 1)
        
        # If the number of CMP_SPEED is 0 we review the contiguous data, if they are different from 0 we add 1 to the previous number
        elif (Ancient_wasp_engine3.loc[i-1, "CMP_SPEED"] > 0) and (Ancient_wasp_engine3.loc[i, "CMP_SPEED"] == 0) and (Ancient_wasp_engine3.loc[i+1, "CMP_SPEED"] > 0):
            HOURS.append(HOURS[-1] + 1)
        
        # When the number of CMP_SPEED is 0 and the neighbors are 0
        else:
            HOURS.append(HOURS[-1] + 0)
    
    # condition for the end of the dataframe when the value of CMP_SPEED is 0
    except KeyError:
        HOURS.append(HOURS[-1] + 0)

In [10]:
# sanity check of the total of hours
HOURS[8760]

6934

## Creating the deliverable dataframe

In [11]:
# empty dataframe to add deliverable columns
df = pd.DataFrame()

In [12]:
# adding the lists as columns to the dataframe
df["ID"] = ID
df["HOURS"] = HOURS

# using the ID column as an index
df = df.set_index("ID")

In [13]:
# saving it as csv
df.to_csv("hours_challenge.csv")

# Checking the operation of the hour meter

In [14]:
# joining the dataframes to see how the counter behaves
view = pd.concat([Ancient_wasp_engine3, df], axis=1)

Finding sections where CMP_SPEED changes from 0 to values greater than 0

In [15]:
# verifying that when it is 0 it does not increase the counter
view[view["CMP_SPEED"] == 0].head()

,Unnamed: 0,CMP_SPEED,POWER,FUEL_FLOW,CO2,HOURS
506,22/01/2021 02:00,0.0,NaN,NaN,NaN,505
507,22/01/2021 03:00,0.0,NaN,NaN,NaN,505
508,22/01/2021 04:00,0.0,NaN,NaN,NaN,505
509,22/01/2021 05:00,0.0,NaN,NaN,NaN,505
510,22/01/2021 06:00,0.0,NaN,NaN,NaN,505


In [16]:
# to see all rows
pd.set_option('display.max_rows', None)

In [17]:
# reviewing behavior on value change
view.iloc[495:530]

,Unnamed: 0,CMP_SPEED,POWER,FUEL_FLOW,CO2,HOURS
495,21/01/2021 15:00,5124.705362,4013.477362,0.413487,1.126008,495
496,21/01/2021 16:00,5105.505536,3992.349503,0.412962,1.124578,496
497,21/01/2021 17:00,5087.712927,3960.169250,0.411381,1.120273,497
498,21/01/2021 18:00,5097.181598,3954.176154,0.410678,1.118360,498
499,21/01/2021 19:00,5092.612539,3952.863582,0.411213,1.119816,499
500,21/01/2021 20:00,5088.938479,3961.875200,0.411914,1.121725,500
501,21/01/2021 21:00,5107.311761,3970.134777,0.411855,1.121564,501
502,21/01/2021 22:00,5120.505012,3987.924090,0.412709,1.123891,502
503,21/01/2021 23:00,5092.968057,3945.975558,0.410625,1.118216,503
504,22/01/2021 00:00,5102.613535,3956.523960,0.410913,1.118999,504


In [18]:
# verifying that when it is 0 it does not increase the counter
view.tail()

,Unnamed: 0,CMP_SPEED,POWER,FUEL_FLOW,CO2,HOURS
8756,31/12/2021 20:00,0.0,NaN,NaN,NaN,6934
8757,31/12/2021 21:00,0.0,NaN,NaN,NaN,6934
8758,31/12/2021 22:00,0.0,NaN,NaN,NaN,6934
8759,31/12/2021 23:00,0.0,NaN,NaN,NaN,6934
8760,01/01/2022 00:00,0.0,NaN,NaN,NaN,6934


# Testing

### Testing the code on a different turbine

In [19]:
# using a dataframe starting at 0
engine_metadata[(engine_metadata["PLANT_NAME"] == "MUSTARD-PANDA") & (engine_metadata["ENGINE_ID"] == "ENGINE_7")]

,CUSTOMER_NAME,PLANT_NAME,ENGINE_ID,FILE_ID
108,IDEALISTIC,MUSTARD-PANDA,ENGINE_7,data_109.csv


In [20]:
# Loading dataframe
mustard_panda_engine7 = pd.read_csv("/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_109.csv")

In [21]:
# ID is a list of numbers from 0 to 8760, it corresponds to the number of rows in the dataframe.
ID = [i for i in range(8761)]

# HOURS is an empty list where the number of hours will be added in ascending order.
HOURS = []

In [22]:
# loop through the number of rows
for i in ID:
    
    # Creating a try loop to stop the error that occurs from reaching the end of the dataframe
    try:
        
        # 0 when it is the first row
        if mustard_panda_engine7.index[i] == 0:
            HOURS.append(0)
        # we add 1 to the previous number in case the CMP_SPEED is greater than 0
        elif (mustard_panda_engine7.loc[i, "CMP_SPEED"] > 0):
            HOURS.append(HOURS[-1] + 1)
        
        # If the number of CMP_SPEED is 0 we review the contiguous data, if they are different from 0 we add 1 to the previous number
        elif (mustard_panda_engine7.loc[i-1, "CMP_SPEED"] > 0) and (mustard_panda_engine7.loc[i, "CMP_SPEED"] == 0) and (mustard_panda_engine7.loc[i+1, "CMP_SPEED"] > 0):
            HOURS.append(HOURS[-1] + 1)
        
        # When the number of CMP_SPEED is 0 and the neighbors are 0
        else:
            HOURS.append(HOURS[-1] + 0)
    
    # condition for the end of the dataframe when the value of CMP_SPEED is 0
    except KeyError:
        HOURS.append(HOURS[-1] + 0)

In [23]:
# checking behavior on change from 0 to greater than 0
mustard_panda_engine7["HOURS"] = HOURS
mustard_panda_engine7[600:785]

,Date,CMP_SPEED,POWER,FUEL_FLOW,CO2,HOURS
600,26/01/2021 00:00,4993.694589,4422.559028,0.466481,1.215371,582
601,26/01/2021 01:00,5016.758620,4460.933978,0.468902,1.221679,583
602,26/01/2021 02:00,5025.192677,4487.235173,0.469984,1.224498,584
603,26/01/2021 03:00,5018.028151,4455.748039,0.468528,1.220705,585
604,26/01/2021 04:00,5004.275026,4429.534439,0.466903,1.216472,586
605,26/01/2021 05:00,5008.962260,4430.832969,0.466891,1.216439,587
606,26/01/2021 06:00,5010.896084,4450.362270,0.467779,1.218753,588
607,26/01/2021 07:00,4982.217207,4403.884639,0.466320,1.214952,589
608,26/01/2021 08:00,4991.984203,4394.202974,0.465348,1.212421,590
609,26/01/2021 09:00,4995.899260,4402.475883,0.465797,1.213590,591


In [24]:
mustard_panda_engine7.head()

,Date,CMP_SPEED,POWER,FUEL_FLOW,CO2,HOURS
0,01/01/2021 00:00,0.0,NaN,NaN,NaN,0
1,01/01/2021 01:00,0.0,NaN,NaN,NaN,0
2,01/01/2021 02:00,0.0,NaN,NaN,NaN,0
3,01/01/2021 03:00,0.0,NaN,NaN,NaN,0
4,01/01/2021 04:00,0.0,NaN,NaN,NaN,0


# Simulating a missing value

In [25]:
# Loading dataframe
mustard_panda_engine7 = pd.read_csv("/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_109.csv")

In [26]:
# middle
mustard_panda_engine7.loc[602, "CMP_SPEED"] = 0
mustard_panda_engine7.loc[607, "CMP_SPEED"] = 0

# last
mustard_panda_engine7.loc[8760, "CMP_SPEED"] = 0

In [27]:
# seeing the values 0, on 602 and 607
mustard_panda_engine7[600:610]

,Date,CMP_SPEED,POWER,FUEL_FLOW,CO2
600,26/01/2021 00:00,4993.694589,4422.559028,0.466481,1.215371
601,26/01/2021 01:00,5016.758620,4460.933978,0.468902,1.221679
602,26/01/2021 02:00,0.000000,4487.235173,0.469984,1.224498
603,26/01/2021 03:00,5018.028151,4455.748039,0.468528,1.220705
604,26/01/2021 04:00,5004.275026,4429.534439,0.466903,1.216472
605,26/01/2021 05:00,5008.962260,4430.832969,0.466891,1.216439
606,26/01/2021 06:00,5010.896084,4450.362270,0.467779,1.218753
607,26/01/2021 07:00,0.000000,4403.884639,0.466320,1.214952
608,26/01/2021 08:00,4991.984203,4394.202974,0.465348,1.212421
609,26/01/2021 09:00,4995.899260,4402.475883,0.465797,1.213590


In [28]:
# ID is a list of numbers from 0 to 8760, it corresponds to the number of rows in the dataframe.
ID = [i for i in range(8761)]

# HOURS is an empty list where the number of hours will be added in ascending order.
HOURS = []

In [29]:
# loop through the number of rows
for i in ID:
    
    # Creating a try loop to stop the error that occurs from reaching the end of the dataframe
    try:
        
        # 0 when it is the first row
        if mustard_panda_engine7.index[i] == 0:
            HOURS.append(0)
        # we add 1 to the previous number in case the CMP_SPEED is greater than 0
        elif (mustard_panda_engine7.loc[i, "CMP_SPEED"] > 0):
            HOURS.append(HOURS[-1] + 1)
        
        # If the number of CMP_SPEED is 0 we review the contiguous data, if they are different from 0 we add 1 to the previous number
        elif (mustard_panda_engine7.loc[i-1, "CMP_SPEED"] > 0) and (mustard_panda_engine7.loc[i, "CMP_SPEED"] == 0) and (mustard_panda_engine7.loc[i+1, "CMP_SPEED"] > 0):
            HOURS.append(HOURS[-1] + 1)
        
        # When the number of CMP_SPEED is 0 and the neighbors are 0
        else:
            HOURS.append(HOURS[-1] + 0)
    
    # condition for the end of the dataframe when the value of CMP_SPEED is 0
    except KeyError:
        HOURS.append(HOURS[-1] + 0)

## It works! Add an hour if it is 0 but the neighbors are greater than 0

In [30]:
# Checking that an hour is counted if it is 0 but the contiguous ones are greater than 0
mustard_panda_engine7["HOURS"] = HOURS
mustard_panda_engine7[600:610]

,Date,CMP_SPEED,POWER,FUEL_FLOW,CO2,HOURS
600,26/01/2021 00:00,4993.694589,4422.559028,0.466481,1.215371,582
601,26/01/2021 01:00,5016.758620,4460.933978,0.468902,1.221679,583
602,26/01/2021 02:00,0.000000,4487.235173,0.469984,1.224498,584
603,26/01/2021 03:00,5018.028151,4455.748039,0.468528,1.220705,585
604,26/01/2021 04:00,5004.275026,4429.534439,0.466903,1.216472,586
605,26/01/2021 05:00,5008.962260,4430.832969,0.466891,1.216439,587
606,26/01/2021 06:00,5010.896084,4450.362270,0.467779,1.218753,588
607,26/01/2021 07:00,0.000000,4403.884639,0.466320,1.214952,589
608,26/01/2021 08:00,4991.984203,4394.202974,0.465348,1.212421,590
609,26/01/2021 09:00,4995.899260,4402.475883,0.465797,1.213590,591


In [31]:
# checking the behavior at the end
mustard_panda_engine7.tail()

,Date,CMP_SPEED,POWER,FUEL_FLOW,CO2,HOURS
8756,31/12/2021 20:00,9292.454483,14465.32517,0.889583,2.317723,6238
8757,31/12/2021 21:00,9311.391116,14506.94822,0.891241,2.322043,6239
8758,31/12/2021 22:00,9313.375600,14539.40293,0.891970,2.323944,6240
8759,31/12/2021 23:00,9307.700094,14531.62839,0.892720,2.325897,6241
8760,01/01/2022 00:00,0.000000,14454.89350,0.889603,2.317776,6241


It works and for several cases, not only for the challenge

A function can be created so that by entering only the name of the plant and the type of engine, it can count hours